<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="./images/btp-banner.gif" alt="BTP A&C">
</div>

## Build an Orchestration Workflow using Generative AI Hub SDK
In this demo, we will explore how to build an orchestration workflow using Generative AI Hub SDK. You will learn to combine different modules from orchestration into a pipeline that can be executed with a single API call. Within the pipeline, the response from one module is used as the input for the next module.

## 🎯Learning Objectives
By the end of this demo, you will be able to:
- Describe orchestration feature in **Generative AI Hub**
- Using **Generative AI Hub SDK** to interact with Orchestration services

## 🚨Requirements

Please review the following requirements before starting the demo: 
- You have a running **orchestration deployment** in Generative AI Hub. 

## 💼Use Case: Supplier Risk Analysis
**Business Problem**:
The company, BestRun SE, has launched a new project requiring their supply chain planner to identify the best supplier from several candidates. To support this, the planner gathered supplier data from various sources into a single file. The core challenge lies in evaluating supplier risk using key performance indicators such as reliability, quality, and market trends. This assessment is critical for making a well-informed supplier selection aligned with business goals.

**Proposed Solution**:
The solution analyzes supplier performance by evaluating metrics like quality, pricing, and delivery times while incorporating contextual data such as industry trends and past evaluations. Sensitive information is protected using data masking to anonymize personal details. An orchestration service integrates data ingestion, vector-based grounding, and large language model execution to streamline analysis, helping organizations gain deeper insights into supplier risk and improve procurement strategies.

### Step 1: Install Python Packages  
Run the following package installations. **pip** is the package installer for Python. You can use pip to install packages from the Python Package Index and other indexes. 

**Note:** Jupyter Notebook kernel restart required after package installation. 

In [ ]:
%pip install generative-ai-hub-sdk[all] --break-system-packages
%pip install python-dotenv --break-system-packages

# kernel restart required!!!

⚠️The Python kernel needs to be restarted before continuing. 

> ![](./images/config_001.png)

### Step 2: Initializing the Orchestration Service
After orchestration deployment, you'll receive a unique endpoint URL. You can use the URL to access orchestration service.

In [ ]:
# Generative AI Config
from dotenv import load_dotenv
import os

load_dotenv()  # take environment variables from .env.
# Get the AI Core client ID from environment variables
RESOURCE_GROUP = os.getenv('AICORE_RESOURCE_GROUP')
# Get the Supplier Sourcing API URL from environment variables
YOUR_API_URL = os.getenv('ORCH_DEPLOYMENT_URL')
if not YOUR_API_URL:
    raise ValueError("ORCH_DEPLOYMENT_URL is not set in your .env file.")
print(f"Your Resource Group is {RESOURCE_GROUP}")

### Step 3: Data Grounding
For the purpose of this exercise, this step has been simplified. In a real-world scenario, data preparation can be significantly more complex and time-consuming. The focus here is to highlight the essential steps involved in designing and executing an orchestration flow.

In [ ]:
# --- import json package to read input file ---
import json

# --- Read Supplier Data from File ---
with open('data/supplier_data.json', 'r', encoding='utf-8') as f:
    sample_supplier_data = f.read()
    
# --- Read Grounding Data from File ---
with open('data/grounding_data.json', 'r', encoding='utf-8') as f:
    sample_grounding_data = f.read()

print("✅The data has been imported successfully!")

we simplified text embedding on purpose in this exercise. We will introduce how to properly setup and run embedding in module 2, using SAP HANA Cloud Vector Engine.

In [ ]:
def embed_text(text):
    """
    Simulate text embedding.
    In a real system, this would output a vector representation.
    Here we simply return the text prefixed to indicate it's 'embedded'.
    """
    # The returned 'vector' is just a placeholder string.
    return f"[Embedded Vector Representation of]: {text}"

def retrieve_relevant_context(embedded_grounding, query, top_k=1):
    """
    Simulate a similarity search that retrieves the most relevant grounding context.
    In a real implementation, you would perform a vector similarity search.
    Here we simply return the original embedded grounding data.
    """
    # For simplicity, we assume the entire embedded grounding is relevant.
    return embedded_grounding

# Simulate embedding and retrieval of grounding context 
embedded_grounding = embed_text(sample_grounding_data)
# Simulate a retrieval step based on a query (here we use a dummy query for demonstration)
query = "What are the industry trends and internal evaluations affecting supplier performance?"
relevant_grounding_context = retrieve_relevant_context(embedded_grounding, query)

print("✅The data has been embedded successfully!")

### Step 4: Template Configuration
The code sets up a template for a supplier evaluation assistant. It defines system and user messages to guide the evaluation process. The system message establishes the assistant’s role as an AI specialized in supplier analysis, noting that sensitive information has been masked. The user message includes placeholders for supplier data and contextual grounding, instructing the assistant to analyze and recommend the best supplier based on quality, price, and delivery performance. This structure makes the template ready for use in AI-driven supplier assessment tasks.

In [ ]:
from gen_ai_hub.orchestration.models.template import Template
from gen_ai_hub.orchestration.models.message import SystemMessage, UserMessage
# Define a prompt template that now includes grounding context.
template = Template(
    messages=[
        SystemMessage("You are an AI procurement assistant specialized in supplier evaluation. Note: Sensitive supplier information has been masked."),
        UserMessage(
            "Given the following supplier data and additional grounding context, analyze and recommend the best supplier based on quality, price, and delivery performance. \n\nSupplier Data: {{?supplier_data}}\n\nGrounding Context: {{?grounding_context}}"
        )
    ]
)

print("✅The template has been defined successfully!")

### Step 5: Data masking 
The code maintains data privacy by using Data Masking techniques.  
**Note:**  This step is optional; evaluate whether it is necessary based on the requirements of your particular use case.

In [ ]:
from gen_ai_hub.orchestration.models.data_masking import DataMasking
from gen_ai_hub.orchestration.models.sap_data_privacy_integration import SAPDataPrivacyIntegration, MaskingMethod, ProfileEntity
# Create a proper data masking configuration
data_masking = DataMasking(
    providers=[
        SAPDataPrivacyIntegration(
            method=MaskingMethod.ANONYMIZATION,  # or MaskingMethod.PSEUDONYMIZATION
            entities=[
                ProfileEntity.EMAIL,
                ProfileEntity.PHONE,
                ProfileEntity.PERSON
            ]
        )
    ]
)

print("✅The data masking has been configured successfully!")

### Step 6: Model Configuration
The code imports the LLM class from the gen_ai_hub module, then creates an instance of an advanced language model, specifically "gpt-4o". This instance is configured with the latest version and tailored parameters like maximum token limit and temperature setting to control response variability. It sets the stage for scalable AI-driven text generation tasks.

In [ ]:
from gen_ai_hub.orchestration.models.llm import LLM
# Configure the Language Model (LLM) for processing.
llm = LLM(
    name="gpt-4o",  # Replace with your model name if needed
    version="latest",
    parameters={"max_tokens": 350, "temperature": 0.3}
)

print(f"✅The LLM {llm.name} has been configured successfully!")

### Step 7: Create the Orchestration Configuration
The code here imports the OrchestrationConfig class from the gen_ai_hub.orchestration.models.config module. It then initializes an instance of OrchestrationConfig using the template and LLM variables. This setup is necessary to configure the orchestration process, ensuring the system uses the specified template and language model parameters.

In [ ]:
from gen_ai_hub.orchestration.models.config import OrchestrationConfig
# Create the orchestration configuration with the data masking config.
config = OrchestrationConfig(
    template=template,
    llm=llm,
    data_masking=data_masking  # Use the configuration object instead of a boolean
)

print("✅The orchestration workflow has been constrcuted successfully!")

### Step 8: Run the Orchestration Request
This code leverages the OrchestrationService to run a predefined task using specific template values. It connects to the service through the provided API URL and configuration, executes the task by supplying a text value, and then prints the resulting content. The code ensures streamlined communication with the orchestration system and retrieval of results.

In [ ]:
from gen_ai_hub.orchestration.models.template import TemplateValue
from gen_ai_hub.orchestration.service import OrchestrationService

# Now use YOUR_API_URL to instantiate the orchestration service
orchestration_service = OrchestrationService(api_url=YOUR_API_URL, config=config)

# Run the orchestration workflow by providing both the supplier data and the retrieved grounding context.
result = orchestration_service.run(template_values=[
    TemplateValue(name="supplier_data", value=sample_supplier_data),
    TemplateValue(name="grounding_context", value=relevant_grounding_context)
])
# Print the recommendation output.
print("Supplier Sourcing Recommendation with Data Masking and Grounding (Embedded):")
print(result.orchestration_result.choices[0].message.content)